In [1]:
import requests
import json
import datetime
from pprint import pprint
import pandas as pd
import numpy as np
import gspread

In [2]:
#ローカルにおいてあるconfigファイルを読み出して、認証情報を取得しておく
import sys
sys.path.append("../")
from instaapi.local_config_vook import * 

In [3]:
version = 'v17.0'

In [4]:
def basic_info():
    # 初期
    config = dict()
    #アクセストークン※facebook開発アプリから取得
    config["access_token"]         = ACCESS_TOKEN
    #アプリID※facebook開発アプリから取得
    config["app_id"]               = APP_ID
    #アプリシークレット※facebook開発アプリから取得
    config["app_secret"]           = APP_SECRET
    #インスタグラムビジネスアカウントID※facebook開発アプリから取得
    config['instagram_account_id'] = INSTAGRAM_ACCOUNT_ID
    #APIバージョン
    config["version"]              = version
    # 【修正不要】graphドメイン
    config["graph_domain"]         = 'https://graph.facebook.com/'
    # 【修正不要】エンドポイント
    config["endpoint_base"]        = config["graph_domain"]+config["version"] + '/'
    # 出力
    return config

In [5]:
# リクエスト
params   = basic_info()       # リクエストパラメータ
#response = debugAT(params)    # レスポンス


In [6]:
'''各投稿のinformation(基本的な情報)を取得'''


import requests
#import pandas as pd
#pd.set_option('display.max_rows', None)

# アクセス情報
business_account_id = INSTAGRAM_ACCOUNT_ID
token = ACCESS_TOKEN
username = USER_NAME
fields = 'name,username,biography,follows_count,followers_count,media_count'
media_fields = 'timestamp,permalink,media_url,like_count,comments_count,caption'
period = 'day'


# ユーザー情報を取得する
def user_info(business_account_id=INSTAGRAM_ACCOUNT_ID,token=ACCESS_TOKEN,username=username,fields=fields):
    request_url = "https://graph.facebook.com/"+version+"/{business_account_id}?fields=business_discovery.username({username}){{{fields}}}&access_token={token}".format(business_account_id=business_account_id,username=username,fields=fields,token=token)
#     print(request_url)
    response = requests.get(request_url)
    return response.json()['business_discovery']

#print(user_info(business_account_id,token,username,fields))

# メディア情報を取得する
def user_media_info(business_account_id,token,username,media_fields):
    all_response = []

    request_url = "https://graph.facebook.com/"+version+"/{business_account_id}?fields=business_discovery.username({username}){{media{{{media_fields}}}}}&access_token={token}".format(business_account_id=business_account_id,username=username,media_fields=media_fields,token=token)
#     print(request_url)
    response = requests.get(request_url)
    result = response.json()['business_discovery']
    
    all_response.append(result['media']['data'])
    
    # 過去分がある場合は過去分全て取得する(1度に取得できる件数は25件)
    if 'after' in result['media']['paging']['cursors'].keys():
        next_token = result['media']['paging']['cursors']['after']
        while next_token is not None:
            request_url = "https://graph.facebook.com/"+version+"/{business_account_id}?fields=business_discovery.username({username}){{media.after({next_token}){{{media_fields}}}}}&access_token={token}".format(business_account_id=business_account_id,username=username,media_fields=media_fields,token=token,next_token=next_token)
#             print(request_url)
            response = requests.get(request_url)
            result = response.json()['business_discovery']
            all_response.append(result['media']['data'])
            if 'after' in result['media']['paging']['cursors'].keys():
                next_token = result['media']['paging']['cursors']['after']
            else:
                next_token = None
    
    return all_response

result = user_media_info(business_account_id,token,username,media_fields)

'''結果をデータフレームに格納'''
df_media_info = pd.DataFrame(result[0])
for noc in np.arange(1, len(result)):
    output_per_call = pd.DataFrame(result[noc])
    df_media_info = pd.concat([df_media_info,output_per_call], ignore_index=True)
    
df_media_info


,timestamp,permalink,media_url,like_count,comments_count,caption,id
0,2023-06-17T09:02:13+0000,https://www.instagram.com/p/CtlfJBIP4pn/,https://scontent-itm1-1.cdninstagram.com/v/t51...,25,0,𝗛𝗔𝗡𝗘𝗦\n𝗙𝗥𝗨𝗜𝗧 𝗢𝗙 𝗧𝗛𝗘 𝗟𝗢𝗢𝗠\n𝗥𝗨𝗦𝗦𝗘𝗟𝗟 𝗔𝗧𝗛𝗟𝗘𝗧𝗜𝗖\n𝟭𝟵...,17998169173909267
1,2023-06-12T15:12:28+0000,https://www.instagram.com/p/CtZRibNPPWI/,https://scontent-itm1-1.cdninstagram.com/v/t51...,27,0,𝗖𝗛𝗔𝗠𝗣𝗜𝗢𝗡\n𝗧-𝗦𝗛𝗜𝗥𝗧\n𝟭𝟵𝟱𝟬𝘀〜𝟭𝟵𝟳𝟬𝘀\n年代別タグ一覧\n\n画像出...,17982961961178173
2,2023-06-06T13:47:43+0000,https://www.instagram.com/p/CtJrEeVPgLv/,https://scontent-itm1-1.cdninstagram.com/v/t51...,46,0,𝗟𝗘𝗩𝗜'𝗦\n𝟱𝟬𝟱 𝟲𝟲前期\n\n\\解説記事//\n\n𝟬𝟭\n1973年からリリー...,18059473711410335
3,2023-06-04T15:45:55+0000,https://www.instagram.com/p/CtEvAaGvLe_/,https://scontent-itm1-1.cdninstagram.com/v/t51...,62,1,𝗟𝗘𝗩𝗜'𝗦\n𝟱𝟬𝟱 𝗕𝗜𝗚 𝗘\n\n\\解説記事//\n\n𝟬𝟭\n1967年からリリ...,18181756498275594
4,2023-06-01T13:32:28+0000,https://www.instagram.com/p/Cs8xWY1viji/,https://scontent-itm1-1.cdninstagram.com/v/t51...,50,0,𝗪𝗢𝗥𝗞 𝗕𝗥𝗔𝗡𝗗\n𝟭𝟵𝟯𝟬𝙨〜𝟭𝟵𝟳𝟬𝙨\n年代別タグ一覧\n\n画像出典 \nTHR...,18362833063013100
5,2023-05-28T13:18:12+0000,https://www.instagram.com/p/CsyciYOPlq9/,https://scontent-itm1-1.cdninstagram.com/v/t51...,45,1,𝗨.𝗦.𝗔𝗥𝗠𝗬\n𝗢𝗚-𝟭𝟬𝟳 𝗕𝗔𝗞𝗘𝗥 𝗣𝗔𝗡𝗧𝗦\n\n\\解説記事//\n\n𝟬𝟭...,17899023626736806
6,2023-05-27T00:44:04+0000,https://www.instagram.com/p/CsuhcECPQ8-/,https://scontent-itm1-1.cdninstagram.com/v/t51...,38,1,𝗨.𝗦.𝗔𝗥𝗠𝗬\n𝗠-𝟰𝟳 𝗕𝗔𝗞𝗘𝗥 𝗣𝗔𝗡𝗧𝗦\n\n\\解説記事//\n\n1947...,17956787546605394
7,2023-05-25T13:39:34+0000,https://www.instagram.com/p/CsqwmQcPMvq/,https://scontent-itm1-1.cdninstagram.com/v/t51...,25,0,𝗟𝗲𝘃𝗶'𝘀 \n𝟱𝟲𝟬 𝗟𝗼𝗼𝘀𝗲 𝗙𝗶𝘁\n解説記事\n\n画像出典 \n@furugi...,17954727053465183
8,2023-05-23T15:59:52+0000,https://www.instagram.com/p/Csl3EC1v9V3/,https://scontent-itm1-1.cdninstagram.com/v/t51...,26,1,𝗟𝗲𝘃𝗶'𝘀 \n𝟱𝟱𝟬 𝗥𝗲𝗹𝗮𝘅 𝗙𝗶𝘁\n解説記事\n\n画像出典 \n@furugi...,18364412722017381
9,2023-05-16T13:50:54+0000,https://www.instagram.com/p/CsTmvWTv1z0/,https://scontent-itm1-1.cdninstagram.com/v/t51...,38,0,𝗛𝗘𝗔𝗗 𝗟𝗜𝗚𝗛𝗧\n𝗕𝗜𝗚 𝗬𝗔𝗡𝗞\n𝗙𝗜𝗩𝗘 𝗕𝗥𝗢𝗧𝗛𝗘𝗥 ...\n\n𝟭𝟵𝟯𝟬...,18016437298580127


In [7]:

'''ここで全投稿のmedia idのリストも作っておく'''

list_media_id = []
for noc in np.arange(len(result)):
    for nop in np.arange(len(result[noc])):
        media_id = result[noc][nop]['id']
        list_media_id.append(media_id)
print('最初の10個だけ表示：',list_media_id[:10])
print('全投稿数：',len(list_media_id))

最初の10個だけ表示： ['17998169173909267', '17982961961178173', '18059473711410335', '18181756498275594', '18362833063013100', '17899023626736806', '17956787546605394', '17954727053465183', '18364412722017381', '18016437298580127']
全投稿数： 60


In [8]:
'''投稿ごとのさらに詳細なinsight(reach, saved, impressions, enagegement)を取得する
    ※これは各投稿IDごとにAPIコールが必要なため、処理に少し時間がかかる。改善方法を検討中'''

"""
    ***********************************************************************************
    【APIエンドポイント】
    https://graph.facebook.com/{graph-api-version}/{ig-media-id}/insights?metric={metric}
    ***********************************************************************************
    """
#media IDからインサイトを取得する関数
def media_insight(media_id):
    
    '''リクエスト先のurl作成'''
    metric_for_url = ''
    for mt in metric:
        metric_for_url += mt + '%2C'
    metric_for_url = metric_for_url.rstrip('%2C')
    request_url = params['endpoint_base'] + media_id + '/insights?access_token=' + params[
        'access_token'] + '&metric=' + metric_for_url
    
    response = requests.get(request_url).json()['data']
    response_reshape = dict()
    response_reshape['id'] = media_id
    response_reshape['reach'] = response[0]['values'][0]['value']
    response_reshape['impressions'] = response[1]['values'][0]['value']
    response_reshape['saved'] = response[2]['values'][0]['value']
    response_reshape['engagement'] = response[3]['values'][0]['value']
    
    return response_reshape


metric = ['reach','impressions','saved','engagement']

'''本来は以下のコードだが、一部インスタ側エラーでインサイトを取得できないのでworkaround的に対応する'''
#全てのmediaIDでインサイトを取得する
# results = []
# for nom in np.arange(len(list_media_id)):
#     media_id = list_media_id[nom]
#     out = media_insight(media_id)
#     results.append(out)
    
    
#全てのmediaIDでインサイトを取得する※workaround対応ver
results = []
for nom in np.arange(41):
    media_id = list_media_id[nom]
    out = media_insight(media_id)
    results.append(out)

In [9]:
df_media_insight = pd.DataFrame(results)
df_media_insight

,id,reach,impressions,saved,engagement
0,17998169173909267,126,143,7,32
1,17982961961178173,149,187,7,34
2,18059473711410335,276,337,2,48
3,18181756498275594,338,421,9,72
4,18362833063013100,224,284,4,54
5,17899023626736806,322,387,3,49
6,17956787546605394,296,357,2,41
7,17954727053465183,204,251,0,25
8,18364412722017381,291,358,1,28
9,18016437298580127,254,316,6,44


In [11]:
'''結合してカラムの順番整理'''

df_media = pd.merge(df_media_info,df_media_insight, how='inner', on='id')
df_media = df_media.reindex(columns=[
    'id', 'timestamp','permalink','media_url','like_count','saved','reach','impressions','engagement','caption'])
df_media

,id,timestamp,permalink,media_url,like_count,saved,reach,impressions,engagement,caption
0,17998169173909267,2023-06-17T09:02:13+0000,https://www.instagram.com/p/CtlfJBIP4pn/,https://scontent-itm1-1.cdninstagram.com/v/t51...,25,7,126,143,32,𝗛𝗔𝗡𝗘𝗦\n𝗙𝗥𝗨𝗜𝗧 𝗢𝗙 𝗧𝗛𝗘 𝗟𝗢𝗢𝗠\n𝗥𝗨𝗦𝗦𝗘𝗟𝗟 𝗔𝗧𝗛𝗟𝗘𝗧𝗜𝗖\n𝟭𝟵...
1,17982961961178173,2023-06-12T15:12:28+0000,https://www.instagram.com/p/CtZRibNPPWI/,https://scontent-itm1-1.cdninstagram.com/v/t51...,27,7,149,187,34,𝗖𝗛𝗔𝗠𝗣𝗜𝗢𝗡\n𝗧-𝗦𝗛𝗜𝗥𝗧\n𝟭𝟵𝟱𝟬𝘀〜𝟭𝟵𝟳𝟬𝘀\n年代別タグ一覧\n\n画像出...
2,18059473711410335,2023-06-06T13:47:43+0000,https://www.instagram.com/p/CtJrEeVPgLv/,https://scontent-itm1-1.cdninstagram.com/v/t51...,46,2,276,337,48,𝗟𝗘𝗩𝗜'𝗦\n𝟱𝟬𝟱 𝟲𝟲前期\n\n\\解説記事//\n\n𝟬𝟭\n1973年からリリー...
3,18181756498275594,2023-06-04T15:45:55+0000,https://www.instagram.com/p/CtEvAaGvLe_/,https://scontent-itm1-1.cdninstagram.com/v/t51...,62,9,338,421,72,𝗟𝗘𝗩𝗜'𝗦\n𝟱𝟬𝟱 𝗕𝗜𝗚 𝗘\n\n\\解説記事//\n\n𝟬𝟭\n1967年からリリ...
4,18362833063013100,2023-06-01T13:32:28+0000,https://www.instagram.com/p/Cs8xWY1viji/,https://scontent-itm1-1.cdninstagram.com/v/t51...,50,4,224,284,54,𝗪𝗢𝗥𝗞 𝗕𝗥𝗔𝗡𝗗\n𝟭𝟵𝟯𝟬𝙨〜𝟭𝟵𝟳𝟬𝙨\n年代別タグ一覧\n\n画像出典 \nTHR...
5,17899023626736806,2023-05-28T13:18:12+0000,https://www.instagram.com/p/CsyciYOPlq9/,https://scontent-itm1-1.cdninstagram.com/v/t51...,45,3,322,387,49,𝗨.𝗦.𝗔𝗥𝗠𝗬\n𝗢𝗚-𝟭𝟬𝟳 𝗕𝗔𝗞𝗘𝗥 𝗣𝗔𝗡𝗧𝗦\n\n\\解説記事//\n\n𝟬𝟭...
6,17956787546605394,2023-05-27T00:44:04+0000,https://www.instagram.com/p/CsuhcECPQ8-/,https://scontent-itm1-1.cdninstagram.com/v/t51...,38,2,296,357,41,𝗨.𝗦.𝗔𝗥𝗠𝗬\n𝗠-𝟰𝟳 𝗕𝗔𝗞𝗘𝗥 𝗣𝗔𝗡𝗧𝗦\n\n\\解説記事//\n\n1947...
7,17954727053465183,2023-05-25T13:39:34+0000,https://www.instagram.com/p/CsqwmQcPMvq/,https://scontent-itm1-1.cdninstagram.com/v/t51...,25,0,204,251,25,𝗟𝗲𝘃𝗶'𝘀 \n𝟱𝟲𝟬 𝗟𝗼𝗼𝘀𝗲 𝗙𝗶𝘁\n解説記事\n\n画像出典 \n@furugi...
8,18364412722017381,2023-05-23T15:59:52+0000,https://www.instagram.com/p/Csl3EC1v9V3/,https://scontent-itm1-1.cdninstagram.com/v/t51...,26,1,291,358,28,𝗟𝗲𝘃𝗶'𝘀 \n𝟱𝟱𝟬 𝗥𝗲𝗹𝗮𝘅 𝗙𝗶𝘁\n解説記事\n\n画像出典 \n@furugi...
9,18016437298580127,2023-05-16T13:50:54+0000,https://www.instagram.com/p/CsTmvWTv1z0/,https://scontent-itm1-1.cdninstagram.com/v/t51...,38,6,254,316,44,𝗛𝗘𝗔𝗗 𝗟𝗜𝗚𝗛𝗧\n𝗕𝗜𝗚 𝗬𝗔𝗡𝗞\n𝗙𝗜𝗩𝗘 𝗕𝗥𝗢𝗧𝗛𝗘𝗥 ...\n\n𝟭𝟵𝟯𝟬...


In [12]:
'''直近30日間のアカウントステータスを取得する'''
# ※30日は遡れる日数の上限

import datetime
#今日と28日さかのぼった日付をyyyy/mm/ddの形式で取得する 標準時に設定
delta_days = 30
today = datetime.datetime.utcnow()
date_delta = datetime.datetime.utcnow() - datetime.timedelta(days=delta_days)

yyyymmdd_td = '{yyyy}/{mm}/{dd}'.format(yyyy=today.year,mm=today.month,dd=today.day)
yyyymmdd_delta = '{yyyy}/{mm}/{dd}'.format(yyyy=date_delta.year,mm=date_delta.month,dd=date_delta.day)

print('今日の日付：',yyyymmdd_td)
print('遡った日付：',yyyymmdd_delta)

今日の日付： 2023/6/18
遡った日付： 2023/5/19


In [13]:
# エンドポイントURLの作成
metric = ['follower_count','impressions','profile_views','reach']
metric_for_url = ''
for mt in metric:
    metric_for_url += mt + '%2C'
metric_for_url = metric_for_url.rstrip('%2C')

request_url = params['endpoint_base'] + params['instagram_account_id'] + '/insights?access_token=' + params[
    'access_token'] + '&metric=' + metric_for_url + '&period=day'+ '&since=' + yyyymmdd_delta + '&until=' + yyyymmdd_td
response = requests.get(request_url).json()['data']

In [14]:
list_endtimes = []
list_impressions = []
list_follower_count = []
list_reach = []
list_profile_views = []

for day_n in range(delta_days):
    list_endtimes.append(response[0]['values'][day_n]['end_time'])
    list_follower_count.append(response[0]['values'][day_n]['value'])
    list_impressions.append(response[1]['values'][day_n]['value'])
    list_profile_views.append(response[2]['values'][day_n]['value'])
    list_reach.append(response[3]['values'][day_n]['value'])
    
result = dict()
result['endtime'] = list_endtimes
result['follower_count'] = list_follower_count
result['profile_views'] = list_profile_views
result['impressions'] = list_impressions
result['reach'] = list_reach


In [15]:
#データフレームとして格納
df_account_status_in30days = pd.DataFrame(result)
df_account_status_in30days

#時系列で並び替え
df_account_status_in30days = df_account_status_in30days.sort_values(by='endtime', ascending=False, ignore_index=True)

In [16]:
'''google　ドライブ　スプレッドシートへの出力 ~各投稿のインサイト~'''

json_file = "../instaapi/instagram-insght-vook-dd85f5af7f10.json"
#出力先スプレッドシートの名前
work_book = "instagram_insight"
#出力先シートの名前
work_sheet = "raw"

#(gcpで設定したJsonファイルを指定)
wb = gspread.service_account(filename=json_file)
#ワークブックを選択
sh = wb.open(work_book)

# #シート一覧を取得する
# ws_list = sh.worksheets()

#シートを指定する
ws_raw1 = sh.worksheet(work_sheet)

list_column = ['id', 'timestamp','permalink','media_url','like_count','saved','reach','impressions','engagement']
#カラムを追加
ws_raw1.update('A1:I1', [list_column])
# output_df1 = pd.DataFrame(df_out[list_column]).fillna(0)
output_to_spsheet = df_media[list_column].fillna(0)

#シート変更範囲の指定
value_chenge_pos1 = 'A2:I{}'.format(len(output_to_spsheet)+1)

ws_raw1.update(value_chenge_pos1, output_to_spsheet.to_numpy().tolist())

{'spreadsheetId': '1mCL6IZxBKrueAj-dJIVigYFrgymw2027RieNinnpUZQ',
 'updatedRange': 'raw!A2:I42',
 'updatedRows': 41,
 'updatedColumns': 9,
 'updatedCells': 369}

In [17]:
'''google　ドライブ　スプレッドシートへの出力 ~直近30日のアカウントのインサイト~'''
json_file = "../instaapi/instagram-insght-vook-dd85f5af7f10.json"
#出力先スプレッドシートの名前
work_book = "instagram_insight"
#出力先シートの名前
work_sheet = "raw2"

#(gcpで設定したJsonファイルを指定)
wb = gspread.service_account(filename=json_file)
#ワークブックを選択
sh = wb.open(work_book)

# #シート一覧を取得する
# ws_list = sh.worksheets()

#シートを指定する
ws_raw2 = sh.worksheet(work_sheet)

list_column2 = ['endtime','follower_count','impressions','profile_views','reach']

#カラムを追加
ws_raw2.update('A1:E1', [list_column2])
# output_df1 = pd.DataFrame(df_out[list_column]).fillna(0)
output_to_spsheet = df_account_status_in30days[list_column2].fillna(0)

#シート変更範囲の指定
value_chenge_pos1 = 'A2:E{}'.format(len(output_to_spsheet)+1)

ws_raw2.update(value_chenge_pos1, output_to_spsheet.to_numpy().tolist())

{'spreadsheetId': '1mCL6IZxBKrueAj-dJIVigYFrgymw2027RieNinnpUZQ',
 'updatedRange': "'raw2'!A2:E31",
 'updatedRows': 30,
 'updatedColumns': 5,
 'updatedCells': 150}

In [ ]:
#csvに書き出して保存
today = datetime.datetime.utcnow().strftime('%Y%m%d_%H%M%S')[:8]
df_media.to_csv('../data/output/media_insight_vook_'+today+'.csv')

In [ ]:
#csvに書き出して保存
today = datetime.datetime.utcnow().strftime('%Y%m%d_%H%M%S')[:8]
df_account_status_in30days.to_csv('../data/output/account_status_vook_'+today+'.csv')